In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install --upgrade --force-reinstall numpy gensim

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached smart_open-7.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached wrapt-1.17.2-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.6 MB)
Using cached smart_open-7.1.0-py3-none-any.whl (61 kB)
Using cached wrapt-1.17.2-cp311-cp

In [20]:
import pandas as pd
import numpy as np
import re
import sys
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import os
import time
import gc


import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM,
                                     Bidirectional, LayerNormalization, MultiHeadAttention, Input)


from sklearn.metrics import f1_score
from gensim.models import KeyedVectors

nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)

def is_colab():
  return 'google.colab' in sys.modules

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    stop_words_set = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words_set]
    text = " ".join(tokens)
    return text

MAX_LEN = 100

PAD_TOKEN = "<PAD>"
UNK_TOKEN = "<UNK>"
train_path = "TrainData.csv"
test_path = "TestLabels.csv"
SCRATCH_EMBED_DIM = 128
TRANSFORMER_HEADS = 4
TRANSFORMER_FF_DIM = 256
TRANSFORMER_DROPOUT = 0.1
TRANSFORMER_LR = 3e-4
EPOCHS = 10
BATCH_SIZE = 32
SCRATCH_EMBED_DIM = 128

embedding_binary_flags = {
    'glove': False,
    'fasttext': False,
    'word2vec': True
}
embedding_dims = {
    'glove': 100,
    'fasttext': 300,
    'word2vec': 300,
    'scratch': SCRATCH_EMBED_DIM
}

if is_colab():
  drive_base = "/content/drive/MyDrive/"
  train_path = os.path.join(drive_base, "Shared/IITD/SEM2/AIL721/assign3/TrainData.csv")
  test_path = os.path.join(drive_base, "Shared/IITD/SEM2/AIL721/assign3/TestLabels.csv")
  base_embed_path = os.path.join(drive_base, "gensim_models")
  base_embed_path = "/content/drive/MyDrive/gensim_models"
  embedding_paths = {
      'glove': os.path.join(base_embed_path, 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-100.gz'),
      'fasttext': os.path.join(base_embed_path, 'fasttext-wiki-news-subwords-300', 'fasttext-wiki-news-subwords-300.gz'),
      'word2vec': os.path.join(base_embed_path, 'word2vec-google-news-300', 'word2vec-google-news-300.gz')
  }
else:
  base_embed_path = os.path.expanduser("~/scratch/")
  embedding_paths = {
      'glove': '/home/scai/mtech/aib242286/scratch/glove-wiki-gigaword-100.gz',
      'fasttext': '/home/scai/mtech/aib242286/scratch/fasttext-wiki-news-subwords-300.gz',
      'word2vec': '/home/scai/mtech/aib242286/scratch/word2vec-google-news-300.gz'
  }

# --- 1. Data Loading ---
print("--- 1. Loading Data ---")
try:
    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path, usecols=[1, 2])
except FileNotFoundError as e:
    print(f"ERROR: Could not find data file: {e}")
    print(f"Train path checked: {train_path}")
    print(f"Test path checked: {test_path}")
    sys.exit(1)

df_train.columns = ['Text', 'Label']
df_test.columns = ['Text', 'Label']
df_train.dropna(subset=['Text', 'Label'], inplace=True)
df_test.dropna(subset=['Text', 'Label'], inplace=True)
df_train['Text'] = df_train['Text'].astype(str)
df_test['Text'] = df_test['Text'].astype(str)
print("Train Data Head:")
print(df_train.head())
print("\nTest Data Head:")
print(df_test.head())
print(f"Train data shape after dropna: {df_train.shape}")
print(f"Test data shape after dropna: {df_test.shape}")
print("-" * 30)


# --- 2. Data Preprocessing ---
print("\n--- 2. Data Preprocessing ---")
print("2.a Text Cleaning...")
df_train['Cleaned_Text'] = df_train['Text'].apply(clean_text)
df_test['Cleaned_Text'] = df_test['Text'].apply(clean_text)
print("Sample Cleaned Text (Train):")
print(df_train[['Text', 'Cleaned_Text']].head())

print("\n2.b Tokenization...")
df_train['Tokens'] = df_train['Cleaned_Text'].apply(word_tokenize)
df_test['Tokens'] = df_test['Cleaned_Text'].apply(word_tokenize)
print("Sample Tokenized Text (Train):")
print(df_train[['Cleaned_Text', 'Tokens']].head())

print("\n2.c Vocabulary Creation...")
all_train_tokens = [token for sublist in df_train['Tokens'] for token in sublist]
token_counts = Counter(all_train_tokens)
unique_tokens = sorted(token_counts.keys())

word_to_idx = {PAD_TOKEN: 0, UNK_TOKEN: 1}
current_idx = 2
for token in unique_tokens:
    if token not in word_to_idx:
        word_to_idx[token] = current_idx
        current_idx += 1

idx_to_word = {idx: word for word, idx in word_to_idx.items()}
vocab_size = len(word_to_idx)
unk_idx = word_to_idx[UNK_TOKEN]
pad_idx = word_to_idx[PAD_TOKEN]

print(f"Vocabulary Size (including {PAD_TOKEN} and {UNK_TOKEN}): {vocab_size}")
print(f"Padding Index: {pad_idx}")
print(f"Unknown Index: {unk_idx}")
print("First 20 actual words in vocab:", list(word_to_idx.items())[2:22])

print("\n2.d Label Encoding...")
label_encoder = LabelEncoder()
df_train['Label_Encoded'] = label_encoder.fit_transform(df_train['Label'])
df_test['Label_Encoded'] = label_encoder.transform(df_test['Label'])
num_classes = len(label_encoder.classes_)
print(f"Number of Classes: {num_classes}")
print("Label Mapping:", dict(zip(label_encoder.classes_, range(num_classes))))
print("Sample Encoded Labels (Train):")
print(df_train[['Label', 'Label_Encoded']].head())
print("-" * 30)

# --- 3. TensorFlow Dataset Creation ---
print("\n--- 3. TensorFlow Dataset Creation ---")

def tokens_to_ids(tokens, word_to_idx_map, unk_token_id, max_seq_len):
    token_ids = [word_to_idx_map.get(token, unk_token_id) for token in tokens]
    # Truncate
    token_ids = token_ids[:max_seq_len]
    return tf.cast(token_ids, dtype=tf.int32)

# Create TensorFlow Datasets
train_sequences = [tokens_to_ids(tokens, word_to_idx, unk_idx, MAX_LEN) for tokens in df_train['Tokens'].tolist()]
test_sequences = [tokens_to_ids(tokens, word_to_idx, unk_idx, MAX_LEN) for tokens in df_test['Tokens'].tolist()]

train_labels = df_train['Label_Encoded'].tolist()
test_labels = df_test['Label_Encoded'].tolist()


train_dataset_tf = tf.data.Dataset.from_generator(
    lambda: iter(zip(train_sequences, train_labels)),
    output_signature=(
        tf.TensorSpec(shape=(None,), dtype=tf.int32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)

test_dataset_tf = tf.data.Dataset.from_generator(
    lambda: iter(zip(test_sequences, test_labels)),
    output_signature=(
        tf.TensorSpec(shape=(None,), dtype=tf.int32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)


train_dataset_tf = train_dataset_tf.shuffle(buffer_size=len(df_train))
train_dataset_tf = train_dataset_tf.padded_batch(
    batch_size=BATCH_SIZE,
    padded_shapes=([MAX_LEN], []),
    padding_values=(tf.constant(pad_idx, dtype=tf.int32), tf.constant(0, dtype=tf.int32))
)
train_dataset_tf = train_dataset_tf.prefetch(buffer_size=tf.data.AUTOTUNE)


test_dataset_tf = test_dataset_tf.padded_batch(
    batch_size=BATCH_SIZE,
    padded_shapes=([MAX_LEN], []),
    padding_values=(tf.constant(pad_idx, dtype=tf.int32), tf.constant(0, dtype=tf.int32))
)
test_dataset_tf = test_dataset_tf.prefetch(buffer_size=tf.data.AUTOTUNE)

print(f"Created TensorFlow Train Dataset: {train_dataset_tf}")
print(f"Created TensorFlow Test Dataset: {test_dataset_tf}")


print("\nSample batch from TensorFlow Train Dataset:")
for sequences_batch, labels_batch in train_dataset_tf.take(1):
    print("Sequences batch shape:", sequences_batch.shape)
    print("Labels batch shape:", labels_batch.shape)
    print("First sequence in batch:\n", sequences_batch[0].numpy())
    print("First label in batch:", labels_batch[0].numpy())
print("-" * 30)

print(f"\nParameters for Model (using TensorFlow):")
print(f"Vocabulary Size: {vocab_size}")
print(f"Number of Classes: {num_classes}")
print(f"Padding Index: {pad_idx}")
print(f"Max Sequence Length (for padding): {MAX_LEN}")
print("\nTensorFlow Dataset preparation complete. Ready for model building and training.")


--- 1. Loading Data ---
Train Data Head:
                                                Text     Label
0  worldcom ex-boss launches defence lawyers defe...  business
1  german business confidence slides german busin...  business
2  bbc poll indicates economic gloom citizens in ...  business
3  lifestyle  governs mobile choice  faster  bett...      tech
4  enron bosses in $168m payout eighteen former e...  business

Test Data Head:
                                                Text     Label
0  qpr keeper day heads for preston queens park r...     sport
1  software watching while you work software that...      tech
2  d arcy injury adds to ireland woe gordon d arc...     sport
3  india s reliance family feud heats up the ongo...  business
4  boro suffer morrison injury blow middlesbrough...     sport
Train data shape after dropna: (1490, 2)
Test data shape after dropna: (735, 2)
------------------------------

--- 2. Data Preprocessing ---
2.a Text Cleaning...
Sample Cleaned Text (Tr

## Part 1: The code for experimenting CNN+LSTM

In [21]:
def dataset_to_numpy(tf_dataset):
    X, y = [], []
    for x_batch, y_batch in tf_dataset:
        X.append(x_batch.numpy())
        y.append(y_batch.numpy())
    return np.vstack(X), np.hstack(y)

X_train_np, y_train_np = dataset_to_numpy(train_dataset_tf)
X_test_np, y_test_np = dataset_to_numpy(test_dataset_tf)




def load_gensim_embedding(path, binary=False, limit=None):
    print(f"Attempting to load embeddings from: {path}")
    if not os.path.exists(path):
        print("File not found. Skipping.")
        return None
    try:
        model = KeyedVectors.load_word2vec_format(path, binary=binary, limit=limit)
        print(f"Loaded {len(model.key_to_index)} vectors.")
        return model
    except Exception as e:
        print(f"Error loading embedding file {path}: {e}")
        return None


def build_embedding_matrix(word_index, vocab_size, embedding_dim, pretrained_model):
    if pretrained_model is None:
        print("No pre-trained model provided, embedding matrix not built.")
        return None
    if pretrained_model.vector_size != embedding_dim:
        print(f"ERROR: Dimension mismatch. Pretrained model has dim {pretrained_model.vector_size}, requested {embedding_dim}")
        return None

    print(f"Building embedding matrix with dimension: {embedding_dim}")
    embedding_matrix = np.random.uniform(-0.05, 0.05, (vocab_size, embedding_dim))
    hits = 0
    misses = 0
    for word, i in word_index.items():
        if i >= vocab_size: continue
        if word in pretrained_model:
            embedding_matrix[i] = pretrained_model[word]
            hits += 1
        else:
            misses += 1
    embedding_matrix[0] = np.zeros(embedding_dim)
    print(f"Embedding Matrix Ready: {hits} words found, {misses} words missed.")
    return embedding_matrix




def build_text_model(vocab_size, embedding_dim, max_len, num_classes,
                     embedding_matrix=None, freeze_embeddings=True, use_attention=False,
                     lstm_units=64, dropout_rate=0.2):
    """Builds a text classification model (Example: BiLSTM)."""
    model = Sequential(name=f"BiLSTM_Emb{embedding_dim}_Attn{use_attention}")
    model.add(Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=max_len,
        weights=[embedding_matrix] if embedding_matrix is not None else None,
        trainable=not freeze_embeddings if embedding_matrix is not None else True,
        name="embedding"
    ))

    model.add(Bidirectional(LSTM(lstm_units, return_sequences=use_attention, dropout=dropout_rate), name="bidirectional_lstm"))

    if use_attention:
        model.add(GlobalAveragePooling1D(name="pooling_after_lstm_seq"))
    model.add(Dropout(dropout_rate, name="dropout_layer"))
    model.add(Dense(num_classes, activation=None, name="output_dense")) # Output logits

    return model





experiments = [

    {'name': 'GloVe_Attention', 'embedding_type': 'glove', 'embedding_dim': 100, 'binary': False, 'use_attention': True},
    {'name': 'FastText_Attention', 'embedding_type': 'fasttext', 'embedding_dim': 300, 'binary': False, 'use_attention': True},
    {'name': 'Word2Vec_Attention', 'embedding_type': 'word2vec', 'embedding_dim': 300, 'binary': True, 'use_attention': True},
    {'name': 'Scratch_Attention', 'embedding_type': 'scratch', 'embedding_dim': SCRATCH_EMBED_DIM, 'binary': None, 'use_attention': True},

    {'name': 'GloVe_NoAttention', 'embedding_type': 'glove', 'embedding_dim': 100, 'binary': False, 'use_attention': False},
    {'name': 'FastText_NoAttention', 'embedding_type': 'fasttext', 'embedding_dim': 300, 'binary': False, 'use_attention': False},
    {'name': 'Word2Vec_NoAttention', 'embedding_type': 'word2vec', 'embedding_dim': 300, 'binary': True, 'use_attention': False},
    {'name': 'Scratch_NoAttention', 'embedding_type': 'scratch', 'embedding_dim': SCRATCH_EMBED_DIM, 'binary': None, 'use_attention': False},
]



early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)
callbacks_list = [] #not using callbacks to have better comparison
results_list = []
for config in experiments:
    print(f"\n{'='*10} Running Experiment: {config['name']} {'='*10}")
    start_time = time.time()
    embedding_matrix = None
    freeze_embeddings = False
    if config['embedding_type'] != 'scratch':
        path = embedding_paths.get(config['embedding_type'])
        if not path:
            print(f"ERROR: Embedding path for '{config['embedding_type']}' not found. Skipping.")
            continue
        pretrained_model = load_gensim_embedding(path, binary=config['binary'])
        if pretrained_model:
            embedding_matrix = build_embedding_matrix(
                word_to_idx,
                vocab_size,
                config['embedding_dim'],
                pretrained_model
            )
            if embedding_matrix is not None:
                freeze_embeddings = True
            else:
                 print(f"Failed to build embedding matrix for {config['name']}. Skipping.")
                 continue
        else:
            print(f"Failed to load pretrained model for {config['name']}. Skipping.")
            continue


    model = build_text_model(
        vocab_size=vocab_size,
        embedding_dim=config['embedding_dim'],
        max_len=MAX_LEN,
        num_classes=num_classes,
        embedding_matrix=embedding_matrix,
        freeze_embeddings=freeze_embeddings,
        use_attention=config['use_attention']
        #lstm_units=32
    )

    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss=SparseCategoricalCrossentropy(from_logits=True),
        metrics=[SparseCategoricalAccuracy(name='accuracy')]
    )
    model.summary()
    history = model.fit(
        X_train_np, y_train_np,
        validation_data=(X_test_np, y_test_np),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks_list,
        verbose=2
    )


    print("\nEvaluating...")

    test_loss, test_accuracy = model.evaluate(X_test_np, y_test_np, batch_size=BATCH_SIZE, verbose=0)
    y_pred_logits = model.predict(X_test_np, batch_size=BATCH_SIZE)
    y_pred_classes = np.argmax(y_pred_logits, axis=1)
    test_f1_micro = f1_score(y_test_np, y_pred_classes, average='micro')
    test_f1_macro = f1_score(y_test_np, y_pred_classes, average='macro') # Also useful
    end_time = time.time()
    duration = end_time - start_time

    print(f"\nResults for {config['name']}:")
    print(f"  Test Loss: {test_loss:.4f}")
    print(f"  Test Accuracy: {test_accuracy:.4f}")
    print(f"  Test Micro F1: {test_f1_micro:.4f}")
    print(f"  Test Macro F1: {test_f1_macro:.4f}")
    print(f"  Duration: {duration:.2f} seconds")

    results_list.append({
        'Experiment Name': config['name'],
        'Embedding Type': config['embedding_type'],
        'Embedding Dim': config['embedding_dim'],
        'Attention Used': config['use_attention'],
        'Test Loss': test_loss,
        'Test Accuracy': test_accuracy,
        'Test Micro F1': test_f1_micro,
        'Test Macro F1': test_f1_macro,
        'Training Time (s)': duration,
        'Epochs Run': len(history.history['loss'])
    })


    tf.keras.backend.clear_session()


print("\n\n" + "="*20 + " Final Experiment Results " + "="*20)
results_df = pd.DataFrame(results_list)
print(results_df.to_markdown(index=False))
results_df.to_csv("model_cnnlstm_results.csv", index=False)
print("\nResults saved to model_comparison_results.csv")


========== Running Experiment: GloVe_Attention ==========
Attempting to load embeddings from: /content/drive/MyDrive/gensim_models/glove-wiki-gigaword-100/glove-wiki-gigaword-100.gz
Loaded 400000 vectors.
Building embedding matrix with dimension: 100
Embedding Matrix Ready: 22909 words found, 2323 words missed.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "BiLSTM_Emb100_AttnTrue"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │       2,523,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_lstm (Bidirectional)   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pooling_after_lstm_seq               │ ?                           │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_layer (Dropout)              │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output_dense (Dense)                 │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,523,200 (9.63 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_lstm (Bidirectional)   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_layer (Dropout)              │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output_dense (Dense)                 │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
47/47 - 14s - 291ms/step - accuracy: 0.3544 - loss: 1.4308 - val_accuracy: 0.5320 - val_loss: 1.1245
Epoch 2/10
47/47 - 9s - 194ms/step - accuracy: 0.6987 - loss: 0.8258 - val_accuracy: 0.7061 - val_loss: 0.6339
Epoch 3/10
47/47 - 8s - 163ms/step - accuracy: 0.9262 - loss: 0.2946 - val_accuracy: 0.9184 - val_loss: 0.2816
Epoch 4/10
47/47 - 11s - 237ms/step - accuracy: 0.9161 - loss: 0.2735 - val_accuracy: 0.7891 - val_loss: 0.5180
Epoch 5/10
47/47 - 11s - 234ms/step - accuracy: 0.9651 - loss: 0.1456 - val_accuracy: 0.9211 - val_loss: 0.2821
Epoch 6/10
47/47 - 9s - 189ms/step - accuracy: 0.9899 - loss: 0.0580 - val_accuracy: 0.9401 - val_loss: 0.2160
Epoch 7/10
47/47 - 10s - 210ms/step - accuracy: 0.9987 - loss: 0.0204 - val_accuracy: 0.9415 - val_loss: 0.2129
Epoch 8/10
47/47 - 11s - 229ms/step - accuracy: 0.9973 - loss: 0.0146 - val_accuracy: 0.9429 - val_loss: 0.2011
Epoch 9/10
47/47 - 9s - 197ms/step - accuracy: 0.9987 - loss: 0.0103 - val_accuracy: 0.9483 - val_loss: 0.1

In [ ]:
def load_gensim_embedding(path, binary=False, limit=None):
    print(f"Attempting to load embeddings from: {path}")
    if not os.path.exists(path):
        print(f"File not found: {path}. Skipping.")
        return None
    try:
        model = KeyedVectors.load_word2vec_format(path, binary=binary, limit=limit)
        print(f"Loaded {len(model.key_to_index)} vectors from {os.path.basename(path)}.")
        return model
    except Exception as e:
        print(f"Error loading embedding file {path}: {e}")
        return None

def build_embedding_matrix(word_index, vocab_size, embedding_dim, pretrained_model):
    if pretrained_model is None:
        print("No pre-trained model provided, embedding matrix not built.")
        return None
    if pretrained_model.vector_size != embedding_dim:
        print(f"ERROR: Dimension mismatch. Pretrained model has dim {pretrained_model.vector_size}, requested {embedding_dim}")
        return None
    print(f"Building embedding matrix with dimension: {embedding_dim}")
    embedding_matrix = np.random.uniform(-0.05, 0.05, (vocab_size, embedding_dim)).astype(np.float32)
    hits = 0
    misses = 0
    for word, i in word_index.items():
        if i >= vocab_size: continue
        if i == pad_idx:
            continue
        if word in pretrained_model:
            try:
                embedding_matrix[i] = pretrained_model[word]
                hits += 1
            except KeyError:
                 misses += 1
        else:
            misses += 1
    embedding_matrix[pad_idx] = np.zeros(embedding_dim, dtype=np.float32)
    print(f"Embedding Matrix Ready: Shape {embedding_matrix.shape}. Found {hits} words, missed {misses} words.")
    return embedding_matrix

# --- Model Definitions ---
from gensim.models import KeyedVectors
import gc

class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, dense_dim, dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, dropout=dropout_rate)
        self.dropout1 = Dropout(dropout_rate)
        self.norm1 = LayerNormalization(epsilon=1e-6)
        self.ffn = tf.keras.Sequential([Dense(dense_dim, activation='relu'), Dense(embed_dim)])
        self.dropout2 = Dropout(dropout_rate)
        self.norm2 = LayerNormalization(epsilon=1e-6)

    def call(self, inputs, training=False, attention_mask=None):
        if attention_mask is not None:
            attention_mask = tf.cast(tf.expand_dims(attention_mask, axis=1), tf.float32)
        attn_output = self.attention(query=inputs, value=inputs, key=inputs, attention_mask=attention_mask, training=training)
        out1 = self.norm1(inputs + self.dropout1(attn_output, training=training))
        ffn_output = self.ffn(out1, training=training)
        return self.norm2(out1 + self.dropout2(ffn_output, training=training))

class TransformerClassifier(Model):
    def __init__(self, vocab_size, embed_dim, max_len, num_heads, ff_dim, num_transformer_blocks,
                 num_classes, dropout_rate=0.1, embedding_matrix=None, freeze_embeddings=True,
                 use_positional_emb=True, name="transformer_classifier", **kwargs):
        super().__init__(name=name, **kwargs)
        self.use_positional_emb = use_positional_emb
        self.max_len = max_len
        self.embed_dim = embed_dim
        self.num_classes = num_classes
        self.num_transformer_blocks = num_transformer_blocks

        self.token_embedding = Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=max_len,
                                         weights=[embedding_matrix] if embedding_matrix is not None else None,
                                         trainable=not freeze_embeddings if embedding_matrix is not None else True,
                                         mask_zero=True)
        if use_positional_emb:
            self.pos_embedding = Embedding(input_dim=max_len, output_dim=embed_dim)
        else:
            self.pos_embedding = None

        self.embedding_dropout = Dropout(dropout_rate)
        self.encoder_blocks = [
            TransformerEncoder(embed_dim=embed_dim, num_heads=num_heads, dense_dim=ff_dim, dropout_rate=dropout_rate)
            for _ in range(num_transformer_blocks)
        ]
        self.global_avg_pool = GlobalAveragePooling1D()
        self.final_dropout = Dropout(dropout_rate)
        self.dense_out = Dense(num_classes)

    def call(self, inputs, training=False):
        mask = self.token_embedding.compute_mask(inputs)
        x = self.token_embedding(inputs)
        if self.use_positional_emb and self.pos_embedding:
            positions = tf.range(start=0, limit=self.max_len, delta=1)
            position_embeddings = self.pos_embedding(positions)
            x += position_embeddings
        x = self.embedding_dropout(x, training=training)
        for encoder in self.encoder_blocks:
            x = encoder(x, training=training, attention_mask=mask)
        x = self.global_avg_pool(x)
        x = self.final_dropout(x, training=training)
        return self.dense_out(x)

    def build_graph(self):
        x = Input(shape=(self.max_len,), dtype=tf.int32)
        return Model(inputs=x, outputs=self.call(x))

    def get_config(self):
        config = {
            'vocab_size': self.token_embedding.input_dim,
            'embed_dim': self.embed_dim,
            'max_len': self.max_len,
            'num_heads': self.encoder_blocks[0].num_heads if self.num_transformer_blocks > 0 else 0,
            'ff_dim': self.encoder_blocks[0].dense_dim if self.num_transformer_blocks > 0 else 0,
            'num_transformer_blocks': self.num_transformer_blocks,
            'num_classes': self.num_classes,
            'dropout_rate': self.embedding_dropout.rate,
            'use_positional_emb': self.use_positional_emb,
            'freeze_embeddings': not self.token_embedding.trainable,
            'name': self.name
        }
        return config




# --- Define Transformer Experiments ---
transformer_experiments = [
    {'name': 'Transformer_Scratch_L1_Pos', 'embedding_type': 'scratch', 'num_blocks': 1, 'use_positional_emb': True},
    {'name': 'Transformer_Scratch_L2_Pos', 'embedding_type': 'scratch', 'num_blocks': 2, 'use_positional_emb': True},
    {'name': 'Transformer_Scratch_L3_Pos', 'embedding_type': 'scratch', 'num_blocks': 3, 'use_positional_emb': True},
    {'name': 'Transformer_Scratch_L4_Pos', 'embedding_type': 'scratch', 'num_blocks': 4, 'use_positional_emb': True},
    {'name': 'Transformer_Scratch_L1_noPos', 'embedding_type': 'scratch', 'num_blocks': 1, 'use_positional_emb': False},
    {'name': 'Transformer_Scratch_L2_noPos', 'embedding_type': 'scratch', 'num_blocks': 2, 'use_positional_emb': False},
    {'name': 'Transformer_Scratch_L3_noPos', 'embedding_type': 'scratch', 'num_blocks': 3, 'use_positional_emb': False},
    {'name': 'Transformer_Scratch_L4_noPos', 'embedding_type': 'scratch', 'num_blocks': 4, 'use_positional_emb': False},
]

transformer_results_list = []
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1, restore_best_weights=True)
callbacks_list = [early_stopping]


print("\n=== Starting Transformer Experiments (Optimized for Memory) ===")
for config in transformer_experiments:
    print(f"\n{'='*10} Running Transformer Experiment: {config['name']} {'='*10}")
    start_time = time.time()

    emb_type = config['embedding_type']
    emb_matrix = None
    pretrained_model = None
    emb_dim = embedding_dims[emb_type]
    freeze_embeddings = False
    if emb_type != 'scratch':
        path = embedding_paths.get(emb_type)
        is_binary = embedding_binary_flags.get(emb_type, False)
        if not path:
            print(f"ERROR: Embedding path for '{emb_type}' not found. Skipping.")
            continue

        print(f"\n--- Loading & Building for: {emb_type.upper()} ---")
        pretrained_model = load_gensim_embedding(path, binary=is_binary)
        if pretrained_model:
            emb_matrix = build_embedding_matrix(word_to_idx, vocab_size, emb_dim, pretrained_model)
            if emb_matrix is not None:
                freeze_embeddings = True
                print(f"Successfully built embedding matrix for {emb_type}.")
            else:
                 print(f"ERROR: Failed to build embedding matrix for {emb_type}. Skipping.")
                 del pretrained_model
                 gc.collect()
                 continue
        else:
            print(f"ERROR: Failed to load pretrained model for {emb_type}. Skipping.")
            continue

        print("Releasing Gensim model from memory...")
        del pretrained_model
        gc.collect()
        # ------------------------------------------------------
    else:
        print(f"\n--- Using Scratch Embeddings (Dim: {emb_dim}) ---")
        freeze_embeddings = False


    if emb_dim % TRANSFORMER_HEADS != 0:
        print(f"WARNING: Embedding dimension ({emb_dim}) is not divisible by number of heads ({TRANSFORMER_HEADS}). Skipping.")
        if emb_matrix is not None:
            del emb_matrix
            gc.collect()
        continue

    tf.keras.backend.clear_session()
    model = TransformerClassifier(
        vocab_size=vocab_size,
        embed_dim=emb_dim,
        max_len=MAX_LEN,
        num_heads=TRANSFORMER_HEADS,
        ff_dim=TRANSFORMER_FF_DIM,
        num_transformer_blocks=config['num_blocks'],
        num_classes=num_classes,
        dropout_rate=TRANSFORMER_DROPOUT,
        embedding_matrix=emb_matrix,
        freeze_embeddings=freeze_embeddings,
        use_positional_emb=config['use_positional_emb'],
        name=config['name']
    )

    model.compile(
        optimizer=Adam(learning_rate=TRANSFORMER_LR),
        loss=SparseCategoricalCrossentropy(from_logits=True),
        metrics=[SparseCategoricalAccuracy(name='accuracy')]
    )
    try:
        model_graph = model.build_graph()
        print("\nModel Summary:")
        model_graph.summary(line_length=120)
    except Exception as e:
        print(f"Could not build graph for summary: {e}. Trying default summary.")
        model.summary()


    print(f"\nTraining {config['name']}...")
    history = model.fit(
        X_train_np, y_train_np,
        validation_data=(X_test_np, y_test_np),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks_list,
        verbose=2
    )

    # 5. Evaluate Model
    print(f"\nEvaluating {config['name']}...")
    test_loss, test_accuracy = model.evaluate(X_test_np, y_test_np, batch_size=BATCH_SIZE*2, verbose=0)

    y_pred_logits = model.predict(X_test_np, batch_size=BATCH_SIZE*2)
    y_pred_classes = np.argmax(y_pred_logits, axis=1)
    test_f1_micro = f1_score(y_test_np, y_pred_classes, average='micro', zero_division=0)
    test_f1_macro = f1_score(y_test_np, y_pred_classes, average='macro', zero_division=0)

    end_time = time.time()
    duration = end_time - start_time

    # 6. Store Results
    epochs_run = len(history.history['loss']) if hasattr(history, 'history') and 'loss' in history.history else 'N/A'
    print(f"\nResults for {config['name']}:")
    print(f"  Test Loss: {test_loss:.4f}")
    print(f"  Test Accuracy: {test_accuracy:.4f}")
    print(f"  Test Micro F1: {test_f1_micro:.4f}")
    print(f"  Test Macro F1: {test_f1_macro:.4f}")
    print(f"  Duration: {duration:.2f} seconds")
    print(f"  Epochs Run: {epochs_run}")

    transformer_results_list.append({
        'Experiment Name': config['name'],
        'Model Type': 'Transformer',
        'Embedding Type': emb_type,
        'Embedding Dim': emb_dim,
        'Num Blocks': config['num_blocks'],
        'Positional Emb': config['use_positional_emb'],
        'Test Loss': test_loss,
        'Test Accuracy': test_accuracy,
        'Test Micro F1': test_f1_micro,
        'Test Macro F1': test_f1_macro,
        'Training Time (s)': duration,
        'Epochs Run': epochs_run
    })

    # --- Clean up memory for this iteration ---
    print("Cleaning up model and embedding matrix from memory...")
    del model
    if emb_matrix is not None:
        del emb_matrix
    tf.keras.backend.clear_session()
    gc.collect()
    print("Cleanup complete for this iteration.")
    # ------------------------------------------

print("\n=== Finished Transformer Experiments ===")
print("\n\n" + "="*20 + " Final Experiment Results " + "="*20)
all_results = transformer_results_list
if not all_results:
    print("No Transformer experiments were successfully completed.")
else:
    results_df = pd.DataFrame(all_results)
    results_df = results_df.sort_values(by='Test Micro F1', ascending=False)
    print(results_df.to_markdown(index=False))
    results_filename = "transformer_model_comparison_results_mem_opt_2.csv"
    try:
        results_df.to_csv(results_filename, index=False)
        print(f"\nResults saved to {results_filename}")
    except Exception as e:
        print(f"\nError saving results to CSV: {e}")

print("\nScript finished.")


=== Starting Transformer Experiments (Optimized for Memory) ===

========== Running Transformer Experiment: Transformer_Scratch_L1_Pos ==========

--- Using Scratch Embeddings (Dim: 128) ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Model Summary:


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Layer (type)                      ┃ Output Shape                 ┃           Param # ┃ Connected to              
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ input_layer (InputLayer)          │ (None, 100)                  │                 0 │ -                         
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ embedding (Embedding)             │ (None, 100, 128)             │         3,229,696 │ input_layer[0][0]         
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ add (Add)                         │ (None, 100, 128)             │                 0 │ embedding[0][0]           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ dropout (Dropout)                 │ (None, 100, 128)             │                 0 │ add[0][0]                 
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ not_equal (NotEqual)              │ (None, 100)                  │                 0 │ input_layer[0][0]         
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ transformer_encoder               │ (None, 100, 128)             │           330,240 │ dropout[0][0],            
│ (TransformerEncoder)              │                              │                   │ not_equal[0][0]           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ global_average_pooling1d          │ (None, 128)                  │                 0 │ transformer_encoder[0][0] 
│ (GlobalAveragePooling1D)          │                              │                   │                           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ dropout_3 (Dropout)               │ (None, 128)                  │                 0 │ global_average_pooling1d[0
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ dense_2 (Dense)                   │ (None, 5)                    │               645 │ dropout_3[0][0]           
└───────────────────────────────────┴──────────────────────────────┴───────────────────┴───────────────────────────

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Layer (type)                      ┃ Output Shape                 ┃           Param # ┃ Connected to              
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ input_layer (InputLayer)          │ (None, 100)                  │                 0 │ -                         
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ embedding (Embedding)             │ (None, 100, 128)             │         3,229,696 │ input_layer[0][0]         
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ add (Add)                         │ (None, 100, 128)             │                 0 │ embedding[0][0]           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ dropout (Dropout)                 │ (None, 100, 128)             │                 0 │ add[0][0]                 
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ not_equal (NotEqual)              │ (None, 100)                  │                 0 │ input_layer[0][0]         
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ transformer_encoder               │ (None, 100, 128)             │           330,240 │ dropout[0][0],            
│ (TransformerEncoder)              │                              │                   │ not_equal[0][0]           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ transformer_encoder_1             │ (None, 100, 128)             │           330,240 │ transformer_encoder[0][0],
│ (TransformerEncoder)              │                              │                   │ not_equal[0][0]           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ transformer_encoder_2             │ (None, 100, 128)             │           330,240 │ transformer_encoder_1[0][0
│ (TransformerEncoder)              │                              │                   │ not_equal[0][0]           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ transformer_encoder_3             │ (None, 100, 128)             │           330,240 │ transformer_encoder_2[0][0
│ (TransformerEncoder)              │                              │                   │ not_equal[0][0]           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ global_average_pooling1d          │ (None, 128)                  │                 0 │ transformer_encoder_3[0][0
│ (GlobalAveragePooling1D)          │                              │                   │                           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ dropout_9 (Dropout)               │ (None, 128)                  │                 0 │ global_average_pooling1d[0
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ dense_8 (Dense)                   │ (None, 5)                    │               645 │ dropout_9[0][0]           
└───────────────────────────────────┴──────────────────────────────┴───────────────────┴───────────────────────────

 Total params: 4,551,301 (17.36 MB)

 Trainable params: 4,551,301 (17.36 MB)

 Non-trainable params: 0 (0.00 B)


Training Transformer_Scratch_L4_Pos...
Epoch 1/10
47/47 - 85s - 2s/step - accuracy: 0.5389 - loss: 1.1859 - val_accuracy: 0.8612 - val_loss: 0.4072
Epoch 2/10
47/47 - 81s - 2s/step - accuracy: 0.9718 - loss: 0.0847 - val_accuracy: 0.9442 - val_loss: 0.1941
Epoch 3/10
47/47 - 82s - 2s/step - accuracy: 0.9940 - loss: 0.0171 - val_accuracy: 0.9211 - val_loss: 0.3234
Epoch 4/10
47/47 - 69s - 1s/step - accuracy: 0.9993 - loss: 0.0025 - val_accuracy: 0.9374 - val_loss: 0.2411
Epoch 5/10
